# Homework 2 - Part 2

So far we have attempted to build our own deep learning framework from scratch in numpy to implement fully connected networks. While it could be fun to continue in this vein and end up with a `DS677Torch/Flow` library, it would take a lot of work for us to be able to run experiments quickly and at a large scale. We will now migrate to using PyTorch (using TensorFlow is similar, especially with the eager mode in v2.0 and above).

Some advantages of using a deep learning framework:
* Our code will now run on GPUs! This will allow our models to train much faster. When using a framework like PyTorch or TensorFlow you can harness the power of the GPU for your own custom neural network architectures without having to write CUDA code directly (which is beyond the scope of this class).
* You will likely use PyTorch or TensorFlow for your project, and this is an opportunity to get familiar with it.
* Implementing some of the basic functionality by hand should have helped you understand the guts of PyTorch/TensorFlow. Now you can iterate on top of that base of knowledge.
* If you are running deep learning code in your next job, this is likely what you'll be using.


### PyTorch Resources

Justin Johnson has made an excellent [tutorial](https://github.com/jcjohnson/pytorch-examples) for PyTorch.

You can also find the detailed [API doc](http://pytorch.org/docs/stable/index.html) here. If you have other questions that are not addressed by the API docs, the [PyTorch forum](https://discuss.pytorch.org/) is a much better place to ask than StackOverflow.

## Part 2 - Implementing Models in PyTorch

In this assignment, you will learn PyTorch on **three different levels of abstraction**, which will help you understand it better and prepare you for the final project.

1. Barebones PyTorch: **Abstraction level 1**, we will work directly with the lowest-level PyTorch Tensors and use the Functional API to implement layers and networks.
2. PyTorch Module API: **Abstraction level 2**, we will use `nn.Module` to define arbitrary neural network architectures. This is the version you will commonly see in research/prototyping code (this is how I write networks in my experiments, for instance). This is flexible enough to implement arbitrary architectures while affording some convenience.
3. PyTorch Sequential API: **Abstraction level 3**, we will use `nn.Sequential` to define a linear feed-forward network very conveniently.

Finally, after going through the above exercises, you will attempt an open-ended challenge on CIFAR-10. Please implement your own network (avoid pre-defined models in the PyTorch library) to get as high accuracy as possible on CIFAR-10. You can experiment with any layer, optimizer, hyperparameters or other advanced features.

(This homework is adapted from the Stanford CS231n homework assignment.)

## GPU

You can manually switch to a GPU device on Colab by clicking `Runtime -> Change runtime type` and selecting `GPU` under `Hardware Accelerator`. You should do this before running the following cells to import packages, since the kernel gets restarted upon switching runtimes. Alternatively, if you have access to a GPU some other way, you may use that resource. It is strongly recommended that you use a GPU since it allows you to run experiments on CIFAR-10 much quicker than you would on a CPU.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

USE_GPU = True
dtype = torch.float32 # We will be using float throughout this tutorial.

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss.
print_every = 100
print('using device:', device)

using device: cuda


### Load CIFAR-10

Now, let's load the CIFAR-10 dataset. Since the files will be deleted if you close a runtime, you may have to download the dataset again when using Colab. You may modify the code to save the dataset to your Google Drive and mount the dataset from the drive, but if you don't do that - it should still be relatively painless.

In [2]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64,
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64,
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True,
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

100%|██████████| 170498071/170498071 [00:10<00:00, 15762710.47it/s]


Extracting ./cs231n/datasets/cifar-10-python.tar.gz to ./cs231n/datasets
Files already downloaded and verified
Files already downloaded and verified


## Pytorch HW starts here

### Barebones PyTorch

PyTorch ships with high-level APIs to help us define model architectures conveniently, which we will cover in Part II of this tutorial. In this section, we will start with the barebone PyTorch elements to understand the autograd engine better. After this exercise, you will come to appreciate the high-level model API more.

We will start with a simple fully-connected ReLU network with two hidden layers and no biases for CIFAR classification.
This implementation computes the forward pass using operations on PyTorch Tensors, and uses PyTorch autograd to compute gradients. It is important that you understand every line, because you will write a ConvNet in a similar way after the example.

When we create a PyTorch Tensor with `requires_grad=True`, then operations involving that Tensor will not just compute values; they will also build up a computational graph in the background, allowing us to easily backpropagate through the graph to compute gradients of some Tensors with respect to a downstream loss. Concretely if x is a Tensor with `x.requires_grad == True` then after backpropagation `x.grad` will be another Tensor holding the gradient of x with respect to the scalar loss at the end.

In [3]:
# Need a flatten function to convert [N, d1, ..., dK] tensors to [N, d1*...*dK] matrices

def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print('Before flattening: ', x)
    print('After flattening: ', flatten(x))

test_flatten()

Before flattening:  tensor([[[[ 0,  1],
          [ 2,  3],
          [ 4,  5]]],


        [[[ 6,  7],
          [ 8,  9],
          [10, 11]]]])
After flattening:  tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])


#### Two Layer Fully Connected Network

Nothing to implement here, but you should be familiar with how this function works.

In [4]:
import torch.nn.functional as F  # useful stateless functions

def two_layer_fc(x, params):
    """
    A fully-connected neural networks; the architecture is:
    NN is fully connected -> ReLU -> fully connected layer.
    Note that this function only defines the forward pass;
    PyTorch will take care of the backward pass for us.

    The input to the network will be a minibatch of data, of shape
    (N, d1, ..., dM) where d1 * ... * dM = D. The hidden layer will have H units,
    and the output layer will produce scores for C classes.

    Inputs:
    - x: A PyTorch Tensor of shape (N, d1, ..., dM) giving a minibatch of
      input data.
    - params: A list [w1, w2] of PyTorch Tensors giving weights for the network;
      w1 has shape (D, H) and w2 has shape (H, C).

    Returns:
    - scores: A PyTorch Tensor of shape (N, C) giving classification scores for
      the input data x.
    """
    # first we flatten the image
    x = flatten(x)  # shape: [batch_size, C x H x W]

    w1, w2 = params

    # Forward pass: compute predicted y using operations on Tensors. Since w1 and
    # w2 have requires_grad=True, operations involving these Tensors will cause
    # PyTorch to build a computational graph, allowing automatic computation of
    # gradients. Since we are no longer implementing the backward pass by hand we
    # don't need to keep references to intermediate values.
    # you can also use `.clamp(min=0)`, equivalent to F.relu()
    x = F.relu(x.mm(w1))
    x = x.mm(w2)
    return x


def two_layer_fc_test():
    hidden_layer_size = 42
    x = torch.zeros((64, 50), dtype=dtype)  # minibatch size 64, feature dimension 50
    w1 = torch.zeros((50, hidden_layer_size), dtype=dtype)
    w2 = torch.zeros((hidden_layer_size, 10), dtype=dtype)
    scores = two_layer_fc(x, [w1, w2])
    print(scores.size())  # you should see [64, 10]

two_layer_fc_test()

torch.Size([64, 10])


#### Three Layer ConvNet (10 points)

Here you will complete the implementation of the function `three_layer_convnet`, which will perform the forward pass of a three-layer convolutional network. Like above, we can immediately test our implementation by passing zeros through the network. The network should have the following architecture:

1. A convolutional layer (with bias) with `channel_1` filters, each with shape `KW1 x KH1`, and zero-padding of two
2. ReLU nonlinearity
3. A convolutional layer (with bias) with `channel_2` filters, each with shape `KW2 x KH2`, and zero-padding of one
4. ReLU nonlinearity
5. Fully-connected layer with bias, producing scores for C classes.

Note that we have **no softmax activation** here after our fully-connected layer: this is because PyTorch's cross entropy loss performs a softmax activation for you, and by bundling that step in makes computation more efficient.

**HINT**: For convolutions: http://pytorch.org/docs/stable/nn.html#torch.nn.functional.conv2d; pay attention to the shapes of convolutional filters!

In [5]:
import torch.nn.functional as F

def three_layer_convnet(x, params):
    """
    Performs the forward pass of a three-layer convolutional network with the
    architecture defined above.

    Inputs:
    - x: A PyTorch Tensor of shape (N, 3, H, W) giving a minibatch of images
    - params: A list of PyTorch Tensors giving the weights and biases for the
      network; should contain the following:
      - conv_w1: PyTorch Tensor of shape (channel_1, 3, KH1, KW1) giving weights
        for the first convolutional layer
      - conv_b1: PyTorch Tensor of shape (channel_1,) giving biases for the first
        convolutional layer
      - conv_w2: PyTorch Tensor of shape (channel_2, channel_1, KH2, KW2) giving
        weights for the second convolutional layer
      - conv_b2: PyTorch Tensor of shape (channel_2,) giving biases for the second
        convolutional layer
      - fc_w: PyTorch Tensor giving weights for the fully-connected layer. Can you
        figure out what the shape should be?
      - fc_b: PyTorch Tensor giving biases for the fully-connected layer. Can you
        figure out what the shape should be?

    Returns:
    - scores: PyTorch Tensor of shape (N, C) giving classification scores for x
    """
    conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b = params
    scores = None

    # YOUR CODE HERE
    # Implement the forward pass for the three-layer ConvNet.

    # Convolutional layer 1 with ReLU
    x = F.conv2d(x, conv_w1, bias=conv_b1, padding=2)
    x = F.relu(x)

    # Convolutional layer  2 with ReLU
    x = F.conv2d(x, conv_w2, bias=conv_b2, padding=1)
    x = F.relu(x)

    # Flattening output for the fully connected layer
    x = flatten(x)

    # Fully connected layer
    scores = x @ fc_w + fc_b

    return scores

In [6]:
# Test your implementation - your output should have shape [64, 10]

def three_layer_convnet_test():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]

    conv_w1 = torch.zeros((6, 3, 5, 5), dtype=dtype)  # [out_channel, in_channel, kernel_H, kernel_W]
    conv_b1 = torch.zeros((6,))  # out_channel
    conv_w2 = torch.zeros((9, 6, 3, 3), dtype=dtype)  # [out_channel, in_channel, kernel_H, kernel_W]
    conv_b2 = torch.zeros((9,))  # out_channel

    # you must calculate the shape of the tensor after two conv layers, before the fully-connected layer
    fc_w = torch.zeros((9 * 32 * 32, 10))
    fc_b = torch.zeros(10)

    scores = three_layer_convnet(x, [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b])
    print(scores.size())  # you should see [64, 10]
three_layer_convnet_test()

torch.Size([64, 10])


#### Initialization and Accuracy methods
Let's write a couple utility methods to initialize the weight matrices for our models. and check the accuracy of our model

- `random_weight(shape)` initializes a weight tensor with the Kaiming normalization method.
- `zero_weight(shape)` initializes a weight tensor with all zeros. Useful for instantiating bias parameters.

The `random_weight` function uses the Kaiming normal initialization method, described in:

He et al, *Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification*, ICCV 2015, https://arxiv.org/abs/1502.01852

In [7]:
def random_weight(shape):
    """
    Create random Tensors for weights; setting requires_grad=True means that we
    want to compute gradients for these Tensors during the backward pass.
    We use Kaiming normalization: sqrt(2 / fan_in)
    """
    if len(shape) == 2:  # FC weight
        fan_in = shape[0]
    else:
        fan_in = np.prod(shape[1:]) # conv weight [out_channel, in_channel, kH, kW]
    # randn is standard normal distribution generator.
    w = torch.randn(shape, device=device, dtype=dtype) * np.sqrt(2. / fan_in)
    w.requires_grad = True
    return w

def zero_weight(shape):
    return torch.zeros(shape, device=device, dtype=dtype, requires_grad=True)

# create a weight of shape [3 x 5]
# you should see the type `torch.cuda.FloatTensor` if you use GPU.
# Otherwise it should be `torch.FloatTensor`
random_weight((3, 5))

tensor([[-0.2520,  0.4642, -0.3804,  0.5539,  0.3622],
        [ 1.0550,  1.1097,  0.6913,  1.3540,  0.2089],
        [-0.6123, -0.8903,  0.9864,  0.4916,  0.3115]], device='cuda:0',
       requires_grad=True)

In [8]:
def check_accuracy_barebones(loader, model_fn, params):
    """
    Check the accuracy of a classification model.

    Inputs:
    - loader: A DataLoader for the data split we want to check
    - model_fn: A function that performs the forward pass of the model,
      with the signature scores = model_fn(x, params)
    - params: List of PyTorch Tensors giving parameters of the model

    Returns: Nothing, but prints the accuracy of the model
    """
    split = 'val' if loader.dataset.train else 'test'
    print('Checking accuracy on the %s set' % split)
    num_correct, num_samples = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.int64)
            scores = model_fn(x, params)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * acc))

#### Training Loop
We can now set up a basic training loop to train our network for one epoch on CIFAR-10. We will train the model using stochastic gradient descent without momentum. We will use `torch.functional.cross_entropy` to compute the loss; you can [read about it here](http://pytorch.org/docs/stable/nn.html#cross-entropy).

The training loop takes as input the neural network function, a list of initialized parameters (`[w1, w2]` in our example), and learning rate.

In [9]:
def train_barebones(model_fn, params, learning_rate):
    """
    Train a model on CIFAR-10.

    Inputs:
    - model_fn: A Python function that performs the forward pass of the model.
      It should have the signature scores = model_fn(x, params) where x is a
      PyTorch Tensor of image data, params is a list of PyTorch Tensors giving
      model weights, and scores is a PyTorch Tensor of shape (N, C) giving
      scores for the elements in x.
    - params: List of PyTorch Tensors giving weights for the model
    - learning_rate: Python scalar giving the learning rate to use for SGD

    Returns: Nothing
    """
    for t, (x, y) in enumerate(loader_train):
        # Move the data to the proper device (GPU or CPU)
        x = x.to(device=device, dtype=dtype)
        y = y.to(device=device, dtype=torch.long)

        # Forward pass: compute scores and loss
        scores = model_fn(x, params)
        loss = F.cross_entropy(scores, y)

        # Backward pass: PyTorch figures out which Tensors in the computational
        # graph has requires_grad=True and uses backpropagation to compute the
        # gradient of the loss with respect to these Tensors, and stores the
        # gradients in the .grad attribute of each Tensor.
        loss.backward()

        # Update parameters. We don't want to backpropagate through the
        # parameter updates, so we scope the updates under a torch.no_grad()
        # context manager to prevent a computational graph from being built.
        with torch.no_grad():
            for w in params:
                w -= learning_rate * w.grad

                # Manually zero the gradients after running the backward pass
                w.grad.zero_()

        if t % print_every == 0:
            print('Iteration %d, loss = %.4f' % (t, loss.item()))
            check_accuracy_barebones(loader_val, model_fn, params)
            print()

#### Train a Two Layer Network
Now we are ready to run the training loop. We need to explicitly allocate tensors for the fully connected weights, `w1` and `w2`.

Each minibatch of CIFAR has 64 examples, so the tensor shape is `[64, 3, 32, 32]`.

After flattening, `x` shape should be `[64, 3 * 32 * 32]`. This will be the size of the first dimension of `w1`.
The second dimension of `w1` is the hidden layer size, which will also be the first dimension of `w2`.

Finally, the output of the network is a 10-dimensional vector that represents the probability distribution over 10 classes.

You don't need to tune any hyperparameters but you should see accuracies above 40% after training for one epoch.

In [10]:
hidden_layer_size = 4000
learning_rate = 1e-2

w1 = random_weight((3 * 32 * 32, hidden_layer_size))
w2 = random_weight((hidden_layer_size, 10))

train_barebones(two_layer_fc, [w1, w2], learning_rate)

Iteration 0, loss = 3.8988
Checking accuracy on the val set
Got 162 / 1000 correct (16.20%)

Iteration 100, loss = 2.1894
Checking accuracy on the val set
Got 332 / 1000 correct (33.20%)

Iteration 200, loss = 2.2370
Checking accuracy on the val set
Got 345 / 1000 correct (34.50%)

Iteration 300, loss = 2.1020
Checking accuracy on the val set
Got 401 / 1000 correct (40.10%)

Iteration 400, loss = 2.2140
Checking accuracy on the val set
Got 403 / 1000 correct (40.30%)

Iteration 500, loss = 1.8544
Checking accuracy on the val set
Got 417 / 1000 correct (41.70%)

Iteration 600, loss = 1.9728
Checking accuracy on the val set
Got 410 / 1000 correct (41.00%)

Iteration 700, loss = 1.4820
Checking accuracy on the val set
Got 421 / 1000 correct (42.10%)



#### Training a ConvNet (5 points)

Use the functions defined above to initialize and train a three layer convolutional network on CIFAR. The network should have the following architecture:

1. Convolutional layer (with bias) with 32 5x5 filters, with zero-padding of 2
2. ReLU
3. Convolutional layer (with bias) with 16 3x3 filters, with zero-padding of 1
4. ReLU
5. Fully-connected layer (with bias) to compute scores for 10 classes

You should initialize your weight matrices using the `random_weight` function defined above, and you should initialize your bias vectors using the `zero_weight` function above.

You don't need to tune any hyperparameters, but if everything works correctly you should achieve an accuracy above 42% after one epoch.

In [11]:
learning_rate = 3e-3

channel_1 = 32
channel_2 = 16

# Convolutional layer 1
conv_w1 = random_weight((channel_1, 3, 5, 5))
conv_b1 = zero_weight((channel_1,))

# Convolutional layer 2
conv_w2 = random_weight((channel_2, channel_1, 3, 3))
conv_b2 = zero_weight((channel_2,))
# Fully-connected layer
# After conv layers, the image size remains 32x32 due to padding, and there are channel_2 feature maps.
fc_w = random_weight((channel_2 * 32 * 32, 10))
fc_b = zero_weight((10,))

params = [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b]
train_barebones(three_layer_convnet, params, learning_rate)


Iteration 0, loss = 3.2043
Checking accuracy on the val set
Got 121 / 1000 correct (12.10%)

Iteration 100, loss = 1.8787
Checking accuracy on the val set
Got 342 / 1000 correct (34.20%)

Iteration 200, loss = 1.8451
Checking accuracy on the val set
Got 397 / 1000 correct (39.70%)

Iteration 300, loss = 1.5734
Checking accuracy on the val set
Got 440 / 1000 correct (44.00%)

Iteration 400, loss = 1.5350
Checking accuracy on the val set
Got 427 / 1000 correct (42.70%)

Iteration 500, loss = 1.3869
Checking accuracy on the val set
Got 458 / 1000 correct (45.80%)

Iteration 600, loss = 1.2341
Checking accuracy on the val set
Got 480 / 1000 correct (48.00%)

Iteration 700, loss = 1.5446
Checking accuracy on the val set
Got 491 / 1000 correct (49.10%)



### PyTorch Module API

Barebone PyTorch requires that we track all the parameter tensors by hand. This is fine for small networks with a few tensors, but it would be extremely inconvenient and error-prone to track tens or hundreds of tensors in larger networks.

PyTorch provides the `nn.Module` API for you to define arbitrary network architectures, while tracking every learnable parameters for you. In Part II, we implemented SGD ourselves. PyTorch also provides the `torch.optim` package that implements all the common optimizers, such as RMSProp, Adagrad, and Adam. It even supports approximate second-order methods like L-BFGS! You can refer to the [doc](http://pytorch.org/docs/master/optim.html) for the exact specifications of each optimizer.

To use the Module API, follow the steps below:

1. Subclass `nn.Module`. Give your network class an intuitive name like `TwoLayerFC`.

2. In the constructor `__init__()`, define all the layers you need as class attributes. Layer objects like `nn.Linear` and `nn.Conv2d` are themselves `nn.Module` subclasses and contain learnable parameters, so that you don't have to instantiate the raw tensors yourself. `nn.Module` will track these internal parameters for you. Refer to the [doc](http://pytorch.org/docs/master/nn.html) to learn more about the dozens of builtin layers. **Warning**: don't forget to call the `super().__init__()` first!

3. In the `forward()` method, define the *connectivity* of your network. You should use the attributes defined in `__init__` as function calls that take tensor as input and output the "transformed" tensor. Do *not* create any new layers with learnable parameters in `forward()`! All of them must be declared upfront in `__init__`.

After you define your Module subclass, you can instantiate it as an object and call it just like the NN forward function in the barebones section.

#### Two Layer Fully Connected Network
Here is a concrete example of a 2-layer fully connected network:

In [12]:
class TwoLayerFC(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        # assign layer objects to class attributes
        self.fc1 = nn.Linear(input_size, hidden_size)
        # nn.init package contains convenient initialization methods
        # http://pytorch.org/docs/master/nn.html#torch-nn-init
        nn.init.kaiming_normal_(self.fc1.weight)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        nn.init.kaiming_normal_(self.fc2.weight)

    def forward(self, x):
        # forward always defines connectivity
        x = flatten(x)
        scores = self.fc2(F.relu(self.fc1(x)))
        return scores

def test_TwoLayerFC():
    input_size = 50
    x = torch.zeros((64, input_size), dtype=dtype)  # minibatch size 64, feature dimension 50
    model = TwoLayerFC(input_size, 42, 10)
    scores = model(x)
    print(scores.size())  # you should see [64, 10]
test_TwoLayerFC()

torch.Size([64, 10])


#### Three Layer ConvNet (10 points)
It's your turn to implement a 3-layer ConvNet followed by a fully connected layer. The network architecture should be the same as in the barebones section:

1. Convolutional layer with `channel_1` 5x5 filters with zero-padding of 2
2. ReLU
3. Convolutional layer with `channel_2` 3x3 filters with zero-padding of 1
4. ReLU
5. Fully-connected layer to `num_classes` classes

You should initialize the weight matrices of the model using the Kaiming normal initialization method.

**HINT**: http://pytorch.org/docs/stable/nn.html#conv2d

After you implement the three-layer ConvNet, the `test_ThreeLayerConvNet` function will run your implementation; it should print `(64, 10)` for the shape of the output scores.

In [13]:
class ThreeLayerConvNet(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()

        # YOUR CODE HERE
        # Set up the layers you need for a three-layer ConvNet with the architecture defined above.
        # Convolutional layer 1
        self.conv1 = nn.Conv2d(in_channel, channel_1, kernel_size=5, padding=2)
        nn.init.kaiming_normal_(self.conv1.weight)

        # Convolutional layer 2
        self.conv2 = nn.Conv2d(channel_1, channel_2, kernel_size=3, padding=1)
        nn.init.kaiming_normal_(self.conv2.weight)

        # Fully connected layer
        self.fc = nn.Linear(channel_2 * 32 * 32, num_classes)
        nn.init.kaiming_normal_(self.fc.weight)

    def forward(self, x):
        scores = None

        # YOUR CODE HERE
        # TODO: Implement the forward function for a 3-layer ConvNet. you should use the layers you defined in __init__ and specify the connectivity of those layers in forward()

        # Conv layer 1 + ReLU
        x = F.relu(self.conv1(x))

        # Conv layer 2+ ReLU
        x = F.relu(self.conv2(x))

        # Flattening  output for the fully-connected layer
        x = flatten(x)

        # Fully connected layer
        scores = self.fc(x)

        return scores


def test_ThreeLayerConvNet():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]
    model = ThreeLayerConvNet(in_channel=3, channel_1=12, channel_2=8, num_classes=10)
    scores = model(x)
    print(scores.size())  # you should see [64, 10]
test_ThreeLayerConvNet()

torch.Size([64, 10])


#### New Accuracy and Training Methods using `torch.optim`

In [14]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

def train(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.

    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for

    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)                                              # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()                                                       # put model to training mode
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with respect to each  parameter of the model.
            loss.backward()

            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy(loader_val, model)
                print()

#### Train a Two Layer Network
Now we are ready to run the training loop. In contrast to barebones, we don't explicitly allocate parameter tensors anymore.

Simply pass the input size, hidden layer size, and number of classes (i.e. output size) to the constructor of `TwoLayerFC`.

You also need to define an optimizer that tracks all the learnable parameters inside `TwoLayerFC`.

You don't need to tune any hyperparameters, but you should see model accuracies above 40% after training for one epoch.

In [15]:
hidden_layer_size = 4000
learning_rate = 1e-2
model = TwoLayerFC(3 * 32 * 32, hidden_layer_size, 10)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train(model, optimizer)

Iteration 0, loss = 3.5798
Checking accuracy on validation set
Got 151 / 1000 correct (15.10)

Iteration 100, loss = 2.6677
Checking accuracy on validation set
Got 319 / 1000 correct (31.90)

Iteration 200, loss = 2.1736
Checking accuracy on validation set
Got 358 / 1000 correct (35.80)

Iteration 300, loss = 2.0966
Checking accuracy on validation set
Got 423 / 1000 correct (42.30)

Iteration 400, loss = 1.7056
Checking accuracy on validation set
Got 416 / 1000 correct (41.60)

Iteration 500, loss = 1.8203
Checking accuracy on validation set
Got 439 / 1000 correct (43.90)

Iteration 600, loss = 1.6495
Checking accuracy on validation set
Got 433 / 1000 correct (43.30)

Iteration 700, loss = 1.5242
Checking accuracy on validation set
Got 432 / 1000 correct (43.20)



#### Train a Three Layer ConvNet (5 points)
You should now use the Module API to train a three layer ConvNet on CIFAR. This should look very similar to training the two-layer network! You don't need to tune any hyperparameters, but you should achieve above above 45% after training for one epoch.

You should train the model using stochastic gradient descent without momentum.

In [16]:
learning_rate = 3e-3
channel_1 = 32
channel_2 = 16

model = None
optimizer = None

# YOUR CODE HERE
# Instantiate your ThreeLayerConvNet model and a corresponding optimizer

model = ThreeLayerConvNet(in_channel=3, channel_1=channel_1, channel_2=channel_2, num_classes=10)

# defining optimizer object for the model's parameters, using SGD
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train(model, optimizer)

Iteration 0, loss = 2.7809
Checking accuracy on validation set
Got 129 / 1000 correct (12.90)

Iteration 100, loss = 1.8454
Checking accuracy on validation set
Got 348 / 1000 correct (34.80)

Iteration 200, loss = 1.8144
Checking accuracy on validation set
Got 401 / 1000 correct (40.10)

Iteration 300, loss = 1.7640
Checking accuracy on validation set
Got 394 / 1000 correct (39.40)

Iteration 400, loss = 1.7881
Checking accuracy on validation set
Got 428 / 1000 correct (42.80)

Iteration 500, loss = 1.5916
Checking accuracy on validation set
Got 446 / 1000 correct (44.60)

Iteration 600, loss = 1.5066
Checking accuracy on validation set
Got 460 / 1000 correct (46.00)

Iteration 700, loss = 1.4663
Checking accuracy on validation set
Got 464 / 1000 correct (46.40)



### PyTorch Sequential API

We just introduced the PyTorch Module API, which allows you to define arbitrary learnable layers and their connectivity.

For simple models like a stack of feed forward layers, you still need to go through 3 steps: subclass `nn.Module`, assign layers to class attributes in `__init__`, and call each layer one by one in `forward()`. Is there a more convenient way? PyTorch must have a `Sequential` container like we had in the previous homeworks that lets us plugin our lego blocks!

Fortunately, PyTorch provides a container Module called `nn.Sequential`, which merges the above steps into one. It is not as flexible as `nn.Module`, because you cannot specify more complex topology than a feed-forward stack, but it's good enough for many use cases.

#### Two Layer Network
Let's see how to rewrite our two layer fully connected network example with `nn.Sequential`, and train it using the training loop defined above.

Again, you don't need to tune any hyperparameters here, but you shoud achieve above 40% accuracy after one epoch of training.

In [17]:
# We need to wrap `flatten` function in a module in order to stack it
# in nn.Sequential
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

hidden_layer_size = 4000
learning_rate = 1e-2

model = nn.Sequential(
    Flatten(),
    nn.Linear(3 * 32 * 32, hidden_layer_size),
    nn.ReLU(),
    nn.Linear(hidden_layer_size, 10),
)

# you can use Nesterov momentum in optim.SGD
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)

train(model, optimizer)

Iteration 0, loss = 2.3364
Checking accuracy on validation set
Got 134 / 1000 correct (13.40)

Iteration 100, loss = 1.5986
Checking accuracy on validation set
Got 359 / 1000 correct (35.90)

Iteration 200, loss = 1.9632
Checking accuracy on validation set
Got 382 / 1000 correct (38.20)

Iteration 300, loss = 1.6596
Checking accuracy on validation set
Got 396 / 1000 correct (39.60)

Iteration 400, loss = 2.2893
Checking accuracy on validation set
Got 437 / 1000 correct (43.70)

Iteration 500, loss = 1.7485
Checking accuracy on validation set
Got 444 / 1000 correct (44.40)

Iteration 600, loss = 1.9323
Checking accuracy on validation set
Got 420 / 1000 correct (42.00)

Iteration 700, loss = 1.2260
Checking accuracy on validation set
Got 445 / 1000 correct (44.50)



#### Three Layer ConvNet (5 points)
Here you should use `nn.Sequential` to define and train a three layer ConvNet with the same architecture we have been using:

1. Convolutional layer (with bias) with 32 5x5 filters, with zero-padding of 2
2. ReLU
3. Convolutional layer (with bias) with 16 3x3 filters, with zero-padding of 1
4. ReLU
5. Fully-connected layer (with bias) to compute scores for 10 classes

You can use the default PyTorch weight initialization.

You should optimize your model using stochastic gradient descent with Nesterov momentum 0.9.

Again, you don't need to tune any hyperparameters but you should see accuracy above 55% after one epoch of training.

In [18]:
channel_1 = 32
channel_2 = 16
learning_rate = 1e-2

model = None
optimizer = None

# YOUR CODE HERE
# Rewrite the 2-layer ConvNet with bias from Part III with the Sequential API.
# Don't forget the optimizer!

# Using nn.Sequential to stack the layers
model = nn.Sequential(
    nn.Conv2d(3, channel_1, kernel_size=5, padding=2),                          # Convolutional layer with 32 5x5 filters, padding 2
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, kernel_size=3, padding=1),                  # Convolutional layer with 16 3x3 filters, padding 1
    nn.ReLU(),
    nn.Flatten(),                                                               # Flattening the output for the fully connected layer
    nn.Linear(channel_2 * 32 * 32, 10),                                         # Fully connected layer with channel_2*32*32 inputs, 10 outputs
)

# Using SGD with Nesterov momentum
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

train(model, optimizer)

Iteration 0, loss = 2.3204
Checking accuracy on validation set
Got 120 / 1000 correct (12.00)

Iteration 100, loss = 1.5563
Checking accuracy on validation set
Got 435 / 1000 correct (43.50)

Iteration 200, loss = 1.6210
Checking accuracy on validation set
Got 474 / 1000 correct (47.40)

Iteration 300, loss = 1.3137
Checking accuracy on validation set
Got 491 / 1000 correct (49.10)

Iteration 400, loss = 1.3565
Checking accuracy on validation set
Got 533 / 1000 correct (53.30)

Iteration 500, loss = 1.1279
Checking accuracy on validation set
Got 550 / 1000 correct (55.00)

Iteration 600, loss = 1.2911
Checking accuracy on validation set
Got 573 / 1000 correct (57.30)

Iteration 700, loss = 1.2029
Checking accuracy on validation set
Got 592 / 1000 correct (59.20)



### CIFAR-10 open-ended challenge (30 points)

In this section, you can experiment with whatever ConvNet architecture you'd like on CIFAR-10.

Now it's your job to experiment with architectures, hyperparameters, loss functions, and optimizers to train a model that achieves **at least 70%** accuracy on the CIFAR-10 **validation** set within 10 epochs. You can use the check_accuracy and train functions from above. You can use either `nn.Module` or `nn.Sequential` API.

Describe what you did at the end of this notebook.

Here are the official API documentation for each component. Note: when using Batch Normalization on images or 2D inputs (with channels), use the method called "BatchNorm2D" in PyTorch.

* Layers in torch.nn package: http://pytorch.org/docs/stable/nn.html
* Activations: http://pytorch.org/docs/stable/nn.html#non-linear-activations
* Loss functions: http://pytorch.org/docs/stable/nn.html#loss-functions
* Optimizers: http://pytorch.org/docs/stable/optim.html


#### Things you might try:
- **Filter size**: Above we used 5x5; would smaller filters be more efficient?
- **Number of filters**: Above we used 32 filters. Do more or fewer do better?
- **Pooling vs Strided Convolution**: Do you use max pooling or just stride convolutions?
- **Batch normalization**: Try adding spatial batch normalization after convolution layers and vanilla batch normalization after affine layers. Do your networks train faster?
- **Network architecture**: The network above has two layers of trainable parameters. Can you do better with a deep network? Good architectures to try include:
    - [conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]
    - [conv-relu-conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]
    - [batchnorm-relu-conv]xN -> [affine]xM -> [softmax or SVM]
- **Global Average Pooling**: Instead of flattening and then having multiple affine layers, perform convolutions until your image gets small (7x7 or so) and then perform an average pooling operation to get to a 1x1 image picture (1, 1 , Filter#), which is then reshaped into a (Filter#) vector. This is used in [Google's Inception Network](https://arxiv.org/abs/1512.00567) (See Table 1 for their architecture).
- **Regularization**: Add l2 weight regularization, or perhaps use Dropout.

#### Tips for training
For each network architecture that you try, you should tune the learning rate and other hyperparameters. When doing this there are a couple important things to keep in mind:

- If the parameters are working well, you should see improvement within a few hundred iterations
- Remember the coarse-to-fine approach for hyperparameter tuning: start by testing a large range of hyperparameters for just a few training iterations to find the combinations of parameters that are working at all.
- Once you have found some sets of parameters that seem to work, search more finely around these parameters. You may need to train for more epochs.
- You should use the validation set for hyperparameter search, and save your test set for evaluating your architecture on the best parameters as selected by the validation set.

#### Going above and beyond
If you are feeling adventurous there are many other features you can implement to try and improve your performance. You are **not required** to implement any of these, but don't miss the fun if you have time!

- Alternative optimizers: you can try Adam, Adagrad, RMSprop, etc.
- Alternative activation functions such as leaky ReLU, parametric ReLU, ELU, or MaxOut.
- Model ensembles
- Data augmentation
- New Architectures
  - [ResNets](https://arxiv.org/abs/1512.03385) where the input from the previous layer is added to the output.
  - [DenseNets](https://arxiv.org/abs/1608.06993) where inputs into previous layers are concatenated together.
  - [This blog has an in-depth overview](https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32)

#### Have fun and happy training!

In [21]:
################################################################################
# Experiment with any architectures, optimizers, and hyperparameters.          #
# Achieve AT LEAST 70% accuracy on the *validation set* within 10 epochs.      #
#                                                                              #
# Note that you can use the check_accuracy function to evaluate on either      #
# the test set or the validation set, by passing either loader_test or         #
# loader_val as the second argument to check_accuracy. You should not touch    #
# the test set until you have finished your architecture and  hyperparameter   #
# tuning, and only run the test set once at the end to report a final value.   #
################################################################################
#model = None
#optimizer = None

# YOUR CODE HERE
# You should get at least 70% accuracy

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data augmentation and normalization for training
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
val_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

batch_size = 128
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Definining CNN Architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 128 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = CNN().to(device)


optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loss fn
criterion = nn.CrossEntropyLoss()

# Training fn
def train(model, optimizer, criterion, epochs=10):
    model.train()
    for epoch in range(epochs):
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimizing
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
        check_accuracy(val_loader, model)

train(model, optimizer, criterion, epochs=10)


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/10], Loss: 1.6582
Checking accuracy on test set
Got 4614 / 10000 correct (46.14)
Epoch [2/10], Loss: 1.1992
Checking accuracy on test set
Got 6009 / 10000 correct (60.09)
Epoch [3/10], Loss: 1.0975
Checking accuracy on test set
Got 6543 / 10000 correct (65.43)
Epoch [4/10], Loss: 0.8172
Checking accuracy on test set
Got 6556 / 10000 correct (65.56)
Epoch [5/10], Loss: 1.0450
Checking accuracy on test set
Got 6885 / 10000 correct (68.85)
Epoch [6/10], Loss: 0.8166
Checking accuracy on test set
Got 7003 / 10000 correct (70.03)
Epoch [7/10], Loss: 0.7320
Checking accuracy on test set
Got 7208 / 10000 correct (72.08)
Epoch [8/10], Loss: 0.7244
Checking accuracy on test set
Got 7296 / 10000 correct (72.96)
Epoch [9/10], Loss: 0.9045
Checking accuracy on test set
Got 7313 / 10000 correct (73.13)
Epoch [10/10], Loss: 0.6467
Checking accuracy on test set
Got 7461 / 10000 correct (74.61)


#### Describe what you did

In the cell below you should write an explanation of what you did, any additional features that you implemented, and/or any graphs that you made in the process of training and evaluating your network.

**Answer:**

The architecture i designed contains many layers. I implemented  a batch normalization, and ReLU activation function layer to better understand the varied dataset. A dropout layer was additionally implemented by me to reduce overfitting that can occur. Apart from that a different set of validation was used during training to ensure that no memorising takes place. I also used Adam optimizer  to adjust learning rates, along with a cross-entropy loss function suitable for classification tasks.



#### Test set -- run this only once

Now that we've gotten a result we're happy with, we test our final model on the test set (which you should store in best_model). Think about how this compares to your validation set accuracy.

In [22]:
best_model = model
check_accuracy(loader_test, best_model)

Checking accuracy on test set
Got 6577 / 10000 correct (65.77)
